In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
%cd '/content/drive/My Drive/University/ProjetML/'
%ls

/content/drive/.shortcut-targets-by-id/1clDDLY0UADr5OF8gCO2UkoIIMu755s4h/ProjetML
1_write_2d_images.ipynb  Data/  Projet/


In [3]:
import pickle
import numpy as np
import pandas as pd
import random
from skimage.exposure import rescale_intensity
import matplotlib.pyplot as plt
%matplotlib inline



### **import of data sets**

In [4]:
with open('Data/Donnees_ENT/Data_sets.pickle', "rb") as f:
  idx_train, idx_test, gt_class, gt_id=pickle.load(f)
f.close() 

    

In [ ]:
with open('Data/Donnees_ENT/red_bandN.pickle', "rb") as fr:
  red_ts_norm=pickle.load(fr)
fr.close()
with open('Data/Donnees_ENT/blue_bandN.pickle', "rb") as fb:
  blue_ts_norm=pickle.load(fb)
fb.close()
with open('Data/Donnees_ENT/green_bandN.pickle', "rb") as fg:
  green_ts_norm=pickle.load(fg)
fg.close()
with open('Data/Donnees_ENT/nir_bandN.pickle', "rb") as fn:
  nir_ts_norm=pickle.load(fn)
fn.close()

In [ ]:
# Lecture des données d'entraînement
train_id_ = gt_id[idx_train]
train_y_ = gt_class[idx_train]
f'échantillons d\'entrainement: {train_y_.shape[0]} pixels, {len(np.unique(train_id_))} objets'

"échantillons d'entrainement: 605431 pixels, 1859 objets"

In [ ]:
# Lecture des données test
test_id = gt_id[idx_test]
f'échantillons test: {test_id.shape[0]} pixels, {len(np.unique(test_id))} objets'

'échantillons test: 207485 pixels, 800 objets'

In [ ]:
# Dataframe pour créer un jeu de validation
samples = pd.DataFrame({'ID':train_id_,'Class':train_y_})
samples = samples.drop_duplicates(keep='first')
samples.head()

,ID,Class
0,422,2
4,2677,5
5,201,3
29,423,2
100,496,2


In [ ]:
# 60,70, et 80% des échantillons de chaque classe affecté au jeu de validation
# Division au niveau objet
valid_60_id = []
train_60id = []
for c in np.unique(samples.Class.values) :
    samples_c = samples.loc[samples.Class==c]
    samples_frac = samples_c.sample(frac=0.6,random_state=1234)
    train_60id.extend( samples_frac.ID.values)
    valid_60_id.extend( samples_c.drop(samples_frac.index).ID.values )

valid_70_id = []
train_70id = []
for c in np.unique(samples.Class.values) :
    samples_c = samples.loc[samples.Class==c]
    samples_frac = samples_c.sample(frac=0.7,random_state=1234)
    train_70id.extend( samples_frac.ID.values)
    valid_70_id.extend( samples_c.drop(samples_frac.index).ID.values )

valid_80_id = []
train_80id = []
for c in np.unique(samples.Class.values) :
    samples_c = samples.loc[samples.Class==c]
    samples_frac = samples_c.sample(frac=0.8,random_state=1234)
    train_80id.extend( samples_frac.ID.values)
    valid_80_id.extend( samples_c.drop(samples_frac.index).ID.values )
print('Test 60%: ',len(train_60id ),len(valid_60_id))
print('Test 70%: ',len(train_70id),len(valid_70_id))
print('Test 80%: ',len(train_80id),len(valid_80_id))



Test 60%:  1115 744
Test 70%:  1301 558
Test 80%:  1487 372


In [ ]:
# Récupérer les positions des nouveaux échantillons d'entraînement et validation
#80%
idx80_train = np.where ( np.isin(gt_id,train_80id) )
idx80_valid = np.where ( np.isin(gt_id,valid_80_id) )

#70%
idx70_train = np.where ( np.isin(gt_id,train_70id) )
idx70_valid = np.where ( np.isin(gt_id,valid_70_id) )

#60%
idx60_train = np.where ( np.isin(gt_id,train_60id) )
idx60_valid = np.where ( np.isin(gt_id,valid_60_id) )

In [ ]:
# Lecture des nouvelles données d'entraînement et de validation
#80
train80_X = np.column_stack ( ( blue_ts_norm[idx80_train], green_ts_norm[idx80_train], red_ts_norm[idx80_train], nir_ts_norm[idx80_train] ) )
train80_y = gt_class[idx80_train]

valid80_X = np.column_stack ( ( blue_ts_norm[idx80_valid], green_ts_norm[idx80_valid], red_ts_norm[idx80_valid], nir_ts_norm[idx80_valid] ) )
valid80_y = gt_class[idx80_valid]

#70
train70_X = np.column_stack ( ( blue_ts_norm[idx70_train], green_ts_norm[idx70_train], red_ts_norm[idx70_train], nir_ts_norm[idx70_train] ) )
train70_y = gt_class[idx70_train]

valid70_X = np.column_stack ( ( blue_ts_norm[idx70_valid], green_ts_norm[idx70_valid], red_ts_norm[idx70_valid], nir_ts_norm[idx70_valid] ) )
valid70_y = gt_class[idx70_valid]

#60
train60_X = np.column_stack ( ( blue_ts_norm[idx60_train], green_ts_norm[idx60_train], red_ts_norm[idx60_train], nir_ts_norm[idx60_train] ) )
train60_y = gt_class[idx60_train]

valid60_X = np.column_stack ( ( blue_ts_norm[idx60_valid], green_ts_norm[idx60_valid], red_ts_norm[idx60_valid], nir_ts_norm[idx60_valid] ) )
valid60_y = gt_class[idx60_valid]

print('set 80: ', train80_X.shape, train80_y.shape, valid80_X.shape, valid80_y.shape)
print('set 70: ', train70_X.shape, train70_y.shape, valid70_X.shape, valid70_y.shape)
print('set 60: ', train60_X.shape, train60_y.shape, valid60_X.shape, valid60_y.shape)


set 80:  (498700, 32) (498700,) (106731, 32) (106731,)
set 70:  (451962, 32) (451962,) (153469, 32) (153469,)
set 60:  (403006, 32) (403006,) (202425, 32) (202425,)


In [ ]:
#data sets

dataS60 = [train60_X, train60_y, valid60_X, valid60_y]
with open("Final_Data_sets60.pickle", "wb") as f:
  pickle.dump(dataS60, f)

f.close()


In [ ]:

dataS70 = [train70_X, train70_y, valid70_X, valid70_y]
with open("Final_Data_sets70.pickle", "wb") as f:
  pickle.dump(dataS70, f)

f.close()


In [ ]:

dataS80 = [train80_X, train80_y, valid80_X, valid80_y]
with open("Final_Data_sets80.pickle", "wb") as f:
  pickle.dump(dataS80, f)

f.close()

Creation set test 


In [43]:
df = pd.read_csv('Data/Donnees_ENT/Test_ID_Label.csv')

In [44]:
df

,ID,Class
0,4,4
1,7,4
2,8,4
3,9,3
4,10,3
...,...,...
795,3659,1
796,3665,1
797,3676,1
798,3749,1


In [ ]:
array = df.values
array

In [ ]:
test_id=array[:,0]
test_class=array[:,1]

In [ ]:
dataS_test = [test_id, test_class]
with open("Final_Data_setsTEST.pickle", "wb") as f:
  pickle.dump(dataS_test, f)

f.close()